In [3]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("dataset/original_dataset.csv")

# Print the total number of rows (data entries) in the dataset
total_data = len(df)
print(f"Total data entries in the dataset: {total_data}")

# Print all column names (attributes) in the dataset
print("\nAttributes (Columns) in the Dataset:")
for col in df.columns:
    print(col)

# Specify the column name containing the fraud category
fraud_column = "fraud category"

# Get the unique classes and their counts
class_counts = df[fraud_column].value_counts()

# Print the results
print("\nFraud Category Counts:")
print(class_counts)


Total data entries in the dataset: 1321

Attributes (Columns) in the Dataset:
assured_age
nominee_relation
occupation
policy sumassured
premium
premiumpaymentmode
annual income
holdermaritalstatus
indiv_requirementflag
policy term
policy payment term
product type
channel
fraud category

Fraud Category Counts:
fraud category
misrepresentation                             939
document tampering                            276
signature forgery                              39
misappropriating funds                         29
misselling                                     15
claims fraud                                   13
impersonation                                   6
logging in business not sourced by oneself      1
kickback                                        1
agent dual pan card                             1
unauthorized activity                           1
Name: count, dtype: int64


In [25]:
from sklearn.preprocessing import LabelEncoder

def preprocess_data(df, target_col):
    df = df.copy()
    
    # Encode categorical columns
    categorical_columns = df.select_dtypes(include=["object"]).columns
    for col in categorical_columns:
        if col != target_col:  # Don't encode the target column here
            df[col] = LabelEncoder().fit_transform(df[col])
    
    # Ensure all columns are numeric
    df = df.apply(pd.to_numeric, errors="coerce")
    df = df.dropna()  # Remove any rows with NaN values after conversion
    return df

# Preprocess data
train_df = preprocess_data(train_df, target_col="Fraud Category")
val_df = preprocess_data(val_df, target_col="Fraud Category")
test_df = preprocess_data(test_df, target_col="Fraud Category")


In [26]:
from sklearn.preprocessing import LabelEncoder

def preprocess_data(df, target_col):
    df = df.copy()
    
    # Encode categorical columns
    categorical_columns = df.select_dtypes(include=["object"]).columns
    for col in categorical_columns:
        if col != target_col:  # Don't encode the target column here
            df[col] = LabelEncoder().fit_transform(df[col])
    
    # Ensure all columns are numeric
    df = df.apply(pd.to_numeric, errors="coerce")
    df = df.dropna()  # Remove any rows with NaN values after conversion
    return df

# Preprocess data
train_df = preprocess_data(train_df, target_col="Fraud Category")
val_df = preprocess_data(val_df, target_col="Fraud Category")
test_df = preprocess_data(test_df, target_col="Fraud Category")


In [27]:
def create_dataset(df, target_col, batch_size=32):
    X = df.drop(columns=[target_col]).values.astype("float32")
    y = df[target_col].values.astype("int32")
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(len(df)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

batch_size = 32
train_dataset = create_dataset(train_df, target_col="Fraud Category", batch_size=batch_size)
val_dataset = create_dataset(val_df, target_col="Fraud Category", batch_size=batch_size)
test_dataset = create_dataset(test_df, target_col="Fraud Category", batch_size=batch_size)


In [1]:
from tensorflow.keras import layers, Model, Sequential

def create_dense_model(input_shape, num_classes):
    model = Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(32, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ])
    return model

# Update input shape and number of classes
input_shape = (train_df.shape[1] - 1,)  # Exclude the target column
num_classes = len(train_df["Fraud Category"].unique())

dense_model = create_dense_model(input_shape, num_classes)
dense_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
dense_model.summary()

# Train Dense Model
dense_model.fit(train_dataset, validation_data=val_dataset, epochs=10)


2025-01-03 17:04:57.066937: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/sujal/anaconda3/envs/tf_gpu/lib/python3.11/site-packages/ipykernel_launcher.py"

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/sujal/anaconda3/envs/tf_gpu/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/sujal/anaconda3/envs/tf_gpu/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/sujal/anaconda3/envs/tf_gpu/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

...


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/sujal/anaconda3/envs/tf_gpu/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/sujal/anaconda3/envs/tf_gpu/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/sujal/anaconda3/envs/tf_gpu/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [29]:
from tensorflow.keras import layers, Sequential

def create_dense_model(input_shape, num_classes):
    model = Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(32, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ])
    return model

# Update input shape and number of classes
input_shape = (train_df.shape[1] - 1,)  # Exclude the target column
num_classes = len(train_df["Fraud Category"].unique())

dense_model_1 = create_dense_model(input_shape, num_classes)
dense_model_1.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
dense_model_1.summary()

# Train Dense Model
dense_model_1.fit(train_dataset, validation_data=val_dataset, epochs=10)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 128)               3072      
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_15 (Dense)            (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 7)                 231       
                                                                 
Total params: 13,639
Trainable params: 13,639
Non-trai

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def save_confusion_matrix(true_labels, predicted_labels, class_names, filename):
    """
    Function to compute and save the confusion matrix as an image.
    """
    # Compute confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels, labels=range(len(class_names)))

    # Plot confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues, xticks_rotation="vertical")

    # Save the figure
    plt.title("Confusion Matrix")
    plt.savefig(filename, bbox_inches="tight")
    plt.close()

# Assuming you have class names
class_names = ["Class_0", "Class_1", "Class_2", "Class_3", "Class_4", "Class_5", "Class_6"]  # Update with actual class names

# Save the confusion matrix
save_confusion_matrix(true_labels.numpy(), ensemble_preds.numpy(), class_names, "confusion_matrix.png")

print("Confusion matrix saved as 'confusion_matrix.png'")


NameError: name 'dense_model' is not defined